## Import from GCS with Pandas 

In [ ]:
import tensorflow as tf
print(tf.version.VERSION)

In [ ]:
import os
import pandas as pd
  
# create an Empty DataFrame object
df = pd.DataFrame()
temp_df = pd.DataFrame()

labels = ['backward', 'forward',
          'land', 'left', 'right', 'takeoff']

for label in labels:
    temp_df = pd.read_csv(f"data/{label}/1.csv", sep=',', comment='%')
    df = pd.concat([df, temp_df])

In [ ]:
import pandas as pd
col = []
for x in range (0, 32):
    x = f'_c{x}'
    col.append(x)
df = pd.DataFrame(columns=col)

## Import From GCS with Spark

In [ ]:
!pip install delta-spark==2.3.0

In [1]:
import pyspark
from delta import *
from pyspark.sql.functions import lit
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.executor.memory", "12g")\
    .config("spark.driver.memory", "12g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/02/14 15:39:50 WARN Utils: Your hostname, Henry-Desktop resolves to a loopback address: 127.0.1.1; using 172.20.208.84 instead (on interface eth0)
24/02/14 15:39:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/henry/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/henry/.ivy2/cache
The jars for the packages stored in: /home/henry/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5a754d00-fcfc-46b8-a9bd-dadbaf30c42f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 67ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

24/02/14 15:39:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


#### Writing to Delta lake

In [ ]:
takeoff = spark.read.options(delimiter='\t').csv("data/takeoff", inferSchema='true')
landdf = spark.read.options(delimiter='\t').csv("data/land", inferSchema='true')
leftdf = spark.read.options(delimiter='\t').csv("data/left", inferSchema='true')
rightdf = spark.read.options(delimiter='\t').csv("data/right", inferSchema='true')
forward = spark.read.options(delimiter='\t').csv("data/forward", inferSchema='true')
backward = spark.read.options(delimiter='\t').csv("data/backward", inferSchema='true')

In [ ]:
takeoff = takeoff.withColumn("label", lit("takeoff"))
landdf = landdf.withColumn("label", lit("land"))
leftdf = leftdf.withColumn("label", lit("left"))
rightdf = rightdf.withColumn("label", lit("right"))
forward = forward.withColumn("label", lit("forward"))
backward = backward.withColumn("label", lit("backward"))

In [ ]:
df = takeoff.union(landdf)
df = df.union(leftdf)
df = df.union(rightdf)
df = df.union(forward)
df = df.union(backward)

In [ ]:
df.write \
  .format("delta") \
  .mode("overwrite") \
  .save("deltalake-table")

In [2]:
df1 = spark.read.format("delta").load("deltalake-table")

### Split into Test Train and Convert to Pandas

In [3]:
pdf = df1.toPandas()

24/02/14 15:40:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
pdf.head(9)

,_c0,label
0,"Sample Index, EXG Channel 0, EXG Channel 1, EX...",forward
1,"0.0,9339.922289839064,40817.81695101463,-56907...",forward
2,"210.0,9383.932874671564,40857.536000911714,-56...",forward
3,"212.0,9356.596691202723,40829.70807906486,-568...",forward
4,"214.0,9377.830848435266,40858.07244277864,-569...",forward
5,"216.0,9342.828016618254,40830.35627965407,-569...",forward
6,"218.0,9357.42370574757,40845.82368681714,-5691...",forward
7,"220.0,9331.56273741278,40818.86748300403,-5690...",forward
8,"222.0,9349.84646437722,40841.263930948255,-569...",forward


In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(pdf, test_size=0.1)

In [6]:
train.head(3)

,_c0,label
1755196,"138.0,29816.131867901317,51920.26369813248,419...",backward
1762041,"200.0,13217.659380156922,18098.0733749954,3224...",backward
1564557,"146.0,41083.26626816586,2004.638553218669,3191...",land


### Train Model with Spark to Pandas

In [7]:
import tensorflow as tf

2024-02-14 15:40:55.329029: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-14 15:40:55.355551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 15:40:55.355585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 15:40:55.356725: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 15:40:55.360908: I tensorflow/core/platform/cpu_feature_guar

In [9]:
!pip install tensorflow_decision_forests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.6 MB/s eta 0:00:0000:0100:01


In [10]:
import tensorflow_decision_forests as tfdf
import pandas as pd

# Load a dataset in a Pandas dataframe.

# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="label")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="label")

2024-02-14 15:41:58.332815: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-14 15:41:58.537015: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-14 15:41:58.537049: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-14 15:41:58.539213: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-14 15:41:58.539250: I external/local_xla/xla/stream_executor

In [11]:
test.shape

(254159, 2)

In [15]:
# Train a Random Forest model.
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

Use /tmp/tmpatqcryi6 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:04.424569. Found 2287430 examples.
Training model...
Model trained in 0:00:16.429663
Compiling model...
Model compiled.


[INFO 24-02-14 15:45:15.7326 CST kernel.cc:1233] Loading model from path /tmp/tmpatqcryi6/model/ with prefix 651c18a55875429d
[INFO 24-02-14 15:45:15.7333 CST decision_forest.cc:660] Model loaded with 300 root(s), 900 node(s), and 1 input feature(s).
[INFO 24-02-14 15:45:15.7333 CST abstract_model.cc:1344] Engine "RandomForestGeneric" built
[INFO 24-02-14 15:45:15.7333 CST kernel.cc:1061] Use fast generic engine


In [14]:
# Summary of the model structure.
model.summary()

# Evaluate the model.
model.evaluate(test_ds)

# Export the model to a SavedModel.
model.save("/home/henry/Avatar/server/model")

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (1):
	c0

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "c0"  1.000000 

Variable Importance: NUM_AS_ROOT:
    1. "c0" 300.000000 

Variable Importance: NUM_NODES:
    1. "c0" 300.000000 

Variable Importance: SUM_SCORE:
    1. "c0" 827.343943 



Winner takes all: true
Out-of-bag evaluation: accuracy:0.174613 logloss:29.7285
Number of trees: 300
Total number of nodes: 900

Number of nodes by tree:
Count: 300 Average: 3 StdDev: 0
Min: 3 Max: 3 Ignored: 0
----------------------------------------------
[ 3, 3] 300 100.00% 100.00% ##########

Depth by leafs:
Count: 600 Average: 1 Std

INFO:tensorflow:Assets written to: /home/henry/Avatar/server/model/assets


### Test Spark Pandas Model

In [ ]:
testdf = spark.read.options(delimiter='\t').csv("gs://bci_bucket/data/takeoff/BrainFlow-RAW_Recordings_takeoff_4.csv", inferSchema='true')

In [ ]:
tdf = testdf.toPandas()

In [ ]:
tdf.head(2)

In [ ]:
!pip show tensorflow_decision_forests

In [ ]:
import pandas as pd
import tensorflow as tf 
import tensorflow_decision_forests as tfdf

In [ ]:
new_model = tf.keras.models.load_model("/home/jovyan/notebooks/modelv3")

In [ ]:
idk = tfdf.keras.pd_dataframe_to_tf_dataset(tdf)

In [ ]:
idk

In [ ]:
labels = ['backward', 'down', 'forward',
          'land', 'left', 'right', 'takeoff', 'up']

In [ ]:
prediction = np.argmax(prediction, 1)

In [ ]:
predicted_label = labels[prediction[0]]
predicted_label